Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label - Whole Body CT TotalSegmentator

***The 104 Anatomies Segmentation Workflow in MONAI Label***

In this notebook, we show the MONAI Label model with the bundle app for whole body CT segmentation. The model is trained with the totalSegmentator dataset [1].

MONAI Label + 3D Slicer integration is used for model training and inference. Whole body CT segmentation is a fundamental task for many downstream workflows. In MONAI Label, we provide a single model for inferencing 104 classes of segments, the model we developed used SegResNet [2] network.

![workflow](./figures/monailabel_wholebody_totalSegmentator_3dslicer/totalsegmentator_monailabel.png)

**Whole Body CT TotalSegmentator Bundle:**

This tutorial uses the whole body ct segmentation bundle. The bundle provides two versions of models, a high-resolution model trained with (1.5 x 1.5 x 1.5)mm images, the other one is the low-resolution model trained with (3.0 x 3.0 x 3.0)mm images. MONAI Label provides functions for visualizing config options, users can select which model to use in the 3D Slicer MONAI Label plugin.

![workflow](./figures/monailabel_wholebody_totalSegmentator_3dslicer/overview_classes.png)

Figure source from the TotalSegmentator [1].

## Setup environment

MONAI Label is pre-built within the MONAI Toolkit. Download monaibundle app

### Prerequisites
- **Install MONAI Label**

In [ ]:
!pip install monailabel

### 1.1 Download monaibundle app
Sample-app with built-in versions are in the Toolkit, pull the monaibundle app using monailabel API 

In [ ]:
!monailabel apps --download --name monaibundle --output apps

### 1.2 Download sample data

The training set is the 104 whole-body structures from the TotalSegmentator released datasets. Users can find more details on the datasets at https://github.com/wasserth/TotalSegmentator. All rights and licenses are reserved for the original authors.

- Target: 104 structures
- Modality: CT
- Source: TotalSegmentator
- Challenge: Large volumes of structures in CT images

Disclaimer: We are not the host of the data. Please make sure to read the requirements and usage policies of the data and give credit to the authors of the dataset! We acknowledge the critical role in the creation of the free publicly available TotalSegmentator Database used in this study.

 #### Preprocessing

 - If you are going to use full dataset of TotalSegmentator, please refer to the dataset link, download the data, create and preprocess the images following [this page](https://zenodo.org/record/6802614).
 
 - In this tutorial, we prepared a sample subset, resampled and ready to use. The subset is only for demonstration. Download [here](https://drive.google.com/file/d/1DtDmERVMjks1HooUhggOKAuDm0YIEunG/view?usp=sharing).
  
 To use the bundle, users need to download the data and merge all annotated labels into one NIFTI file. Each file contains 0-104 values, each value represents one anatomy class.
 
 #### Abdominal CT
 
 - In this tutorial, in order to demo the segmentation results and MONAI Label functions, we provide several automated commands with the MSD Task09 abdominal CT scans. The task is the volumetric (3D) segmentation of the spleen from CT image. The dataset is from the 2018 MICCAI challenge. 
 - Users can also use the original TotalSegmentator sample dataset provided above. 

In [ ]:
!monailabel datasets --download --name Task09_Spleen --output datasets

### 1.3 Starting MONAI Label Server

Specify the bundle name in --conf models <BUNDLENAME> argument. For example: wholeBody_ct_segmentation

In [ ]:
!monailabel start_server --app apps/monaibundle --studies datasets/Task09_Spleen/imagesTs --conf models wholeBody_ct_segmentation

## 2. Auto Segmentation with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for auto segmentation. Pre-trained model is provided by the bundle and loaded by MONAI Label server.


**Start 3D Slicer and MONAI Label Plugin**: the tutorial starts with monaibundle app in the 3D Slicer, please refer to the ["Hello World" tutorial](monailabel_HelloWorld_radiology_3dslicer.ipynb) for detailed instructions on installing 3D Slicer and MONAI Label plugin.

### 2.1 Fetch Data

- Select the bundle model loaded with MONAI Label server, **wholeBody_ct_segmentation** in this tutorial. Start MONAI Label server will automatically download the bundle.


click **next sample** to load image data from datastore.

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody1.png)

### 2.2 High-Resolution and Low-Resolution Models

We retrained two versions of the totalSegmentator models, following the original paper and implementation. To meet multiple demands according to computation resources and performance, we provide a 1.5 mm model and a 3.0 mm model, both models are trained with 104 foreground output channels.

In this bundle, we configured a parameter called `highres`, users can set it to true when using 1.5 mm model, and set it to `false` to use the 3.0 mm model. The high-resolution model is named `model.pt` by default, the low-resolution model is named `model_lowres.pt`.

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody2.png)


### 2.3 Run Auto Inference and Select Config Options

- MONAI Label has loaded the pre-trained weights within the bundle for multi-organ segmentation task, click **run** to do inference on the currently loaded data. 

- Note: the pre-trained models are in the bundle's "**models**" folder. For instance, in this usecase, the downloaded pre-trained model is saved at "**apps/monaibundle/model/wholeBody_ct_segmentation/models**".
- Users can move the axis and slices to view the correct location of segmentation.

Users can monitor the logs in the MONAI Label server terminal

- High Resolution Segmentation Results:

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody3.png)

### 2.4 Edit Annotations and Submit Label

- Users can edit the inference labels manually using MONAI Label segmentation editor, click the **Segment Editor** panel. Users can go back to MONAI Label plugin and click **Submit Label** to save the ground truth label to the file system.

- Note: the final annotations will be saved to "**labels/final**" folder in the study dataset, for example, in this usecase, the ground truth label will be saved to "**datasets/Task09_Spleen/imagesTs/labels/final**"

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody5.png)


## 3. Train and Fine Tune Models

Training and actively fine-tuning models with several iterations is a common labeling scenario. Users can train their models anytime when new annotated labels are saved. Click **Train** button, MONAI Label server will fetch the saved final ground truth label and fine-tune the prior model. 

- Training Config Options:

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody6.png)


Users can monitor the logs in the MONAI Label server terminal

![slicer1](./figures/monailabel_wholebody_totalSegmentator_3dslicer/wholeBody7.png)

Same to the Radiology app, users can do auto segmentation using the latest fine-tuned model. 
The active learning process will select unlabeled batch data.

The next image is selected, already trained images will be marked as labeled data, then won't be selected for the next labeling batch. Active learning strategies such as "first/random" will be used for selecting which unlabeled data to fetch. 



## 4. Repeat Until All Data are Annotated and Trained 

Repeated fetch data and active learning process to section 2.2, until all unlabeled data are annotated and trained. 

For each training loop, the new best metric model will be saved in "**model/model.pt**", in this usecase, ""**apps/monaibundle/wholeBody_ct_segmentation/model/model.pt**" is saved.

## 5. Conclusion

This tutorial demonstrates MONAI Label usage with monaibundle app, the 104 anatomies segmentation with CT scans is introduced. Active learning process, auto inference, segmentation editor, submit labels, and save models are included with the bundle. The monaibundle app highlights the robust model deployment for MONAI Label.

[1] Wasserthal, J., Meyer, M., Breit, H.C., Cyriac, J., Yang, S. and Segeroth, M., 2022. TotalSegmentator: robust segmentation of 104 anatomical structures in CT images. arXiv preprint arXiv:2208.05868.

[2] Myronenko, A., Siddiquee, M.M.R., Yang, D., He, Y. and Xu, D., 2022. Automated head and neck tumor segmentation from 3D PET/CT. arXiv preprint arXiv:2209.10809.